# OpenBnB MCP Sunucusu Entegrasyonu ile Semantic Kernel

Bu not defteri, MCPStdioPlugin kullanarak gerçek OpenBnB MCP sunucusuyla Semantic Kernel'i nasıl kullanacağınızı ve gerçek Airbnb konaklamalarını nasıl arayacağınızı gösterir. LLM erişimi için Azure AI Foundry kullanılır. Ortam değişkenlerinizi ayarlamak için [Kurulum Dersi](/00-course-setup/README.md) bağlantısını takip edebilirsiniz.


## Gerekli Paketleri İçe Aktar


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP Eklenti Bağlantısını Oluşturma

MCPStdioPlugin kullanarak [OpenBnB MCP sunucusuna](https://github.com/openbnb-org/mcp-server-airbnb) bağlanacağız. Bu sunucu, @openbnb/mcp-server-airbnb paketi aracılığıyla Airbnb arama işlevselliği sağlar.


## İstemciyi Oluşturma

Bu örnekte, LLM erişimimiz için Azure AI Foundry kullanacağız. Çevre değişkenlerinizin doğru bir şekilde ayarlandığından emin olun.


## Ortam Yapılandırması

Azure OpenAI ayarlarını yapılandırın. Aşağıdaki ortam değişkenlerinin ayarlandığından emin olun:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP Entegrasyonunu Anlama

Bu not defteri, gerçek Airbnb arama işlevselliği sağlayan **gerçek OpenBnB MCP sunucusuna** bağlanır.

### Nasıl çalışır:

1. **MCPStdioPlugin**: MCP sunucusuyla standart giriş/çıkış iletişimi kullanır
2. **Gerçek NPM Paketi**: `@openbnb/mcp-server-airbnb` paketini npx aracılığıyla indirir ve çalıştırır
3. **Canlı Veri**: Airbnb API'lerinden gerçek mülk verilerini döndürür
4. **Fonksiyon Keşfi**: Aracı, MCP sunucusundan mevcut fonksiyonları otomatik olarak keşfeder

### Mevcut Fonksiyonlar:

OpenBnB MCP sunucusu genellikle şu fonksiyonları sağlar:
- **search_listings** - Konum ve kriterlere göre Airbnb mülklerini arayın
- **get_listing_details** - Belirli mülkler hakkında detaylı bilgi alın
- **check_availability** - Belirli tarihler için müsaitlik kontrolü yapın
- **get_reviews** - Mülkler için yorumları alın
- **get_host_info** - Mülk sahipleri hakkında bilgi alın

### Gereksinimler:

- Sisteminizde kurulu **Node.js**
- MCP sunucu paketini indirmek için **internet bağlantısı**
- **NPX** (Node.js ile birlikte gelir)

### Bağlantıyı Test Etme:

MCP sunucusunu manuel olarak çalıştırarak test edebilirsiniz:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Bu işlem, OpenBnB MCP sunucusunu indirip başlatır ve ardından Semantic Kernel gerçek Airbnb verilerine bağlanır.


## OpenBnB MCP Sunucusu ile Ajanı Çalıştırma

Şimdi, OpenBnB MCP sunucusuna bağlanan ve Stockholm'de 2 yetişkin ve 1 çocuk için gerçek Airbnb konaklamalarını arayan AI Ajanını çalıştıracağız. Arama kriterlerini değiştirmek için `user_inputs` listesini istediğiniz gibi düzenleyebilirsiniz.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Özet  
Tebrikler! Model Context Protocol (MCP) kullanarak gerçek dünya konaklama aramasıyla entegre olan bir yapay zeka ajanı başarıyla oluşturdunuz:

Kullanılan Teknolojiler:  
Semantic Kernel - Azure OpenAI ile akıllı ajanlar oluşturmak için  
Azure AI Foundry - LLM yetenekleri ve sohbet tamamlama için  
MCP (Model Context Protocol) - Standartlaştırılmış araç entegrasyonu için  
OpenBnB MCP Sunucusu - Gerçek Airbnb arama işlevselliği için  
Node.js/NPX - Harici MCP sunucusunu çalıştırmak için  

Neler Öğrendiniz:  
MCP Entegrasyonu: Semantic Kernel ajanlarını harici MCP sunucularına bağlama  
Gerçek Zamanlı Veri Erişimi: Canlı API'ler aracılığıyla gerçek Airbnb mülklerini arama  
Protokol İletişimi: Ajan ve MCP sunucusu arasında stdio iletişimini kullanma  
Fonksiyon Keşfi: MCP sunucularından mevcut fonksiyonları otomatik olarak keşfetme  
Akış Yanıtları: Fonksiyon çağrılarını gerçek zamanlı olarak yakalama ve kaydetme  
HTML İşleme: Ajan yanıtlarını stillendirilmiş tablolar ve etkileşimli görsellerle biçimlendirme  

Sonraki Adımlar:  
Ek MCP sunucuları entegre edin (hava durumu, uçuşlar, restoranlar)  
MCP ve A2A protokollerini birleştiren çoklu ajan sistemi oluşturun  
Kendi veri kaynaklarınız için özel MCP sunucuları oluşturun  
Oturumlar arasında kalıcı sohbet hafızası uygulayın  
Ajanı MCP sunucusu düzenlemesiyle Azure Functions'a dağıtın  
Kullanıcı kimlik doğrulama ve rezervasyon yetenekleri ekleyin  

MCP Mimarisinin Temel Avantajları:  
Standartlaştırma: Yapay zeka ajanlarını harici araçlara bağlamak için evrensel protokol  
Gerçek Zamanlı Veri: Çeşitli hizmetlerden canlı, güncel bilgilere erişim  
Genişletilebilirlik: Yeni veri kaynakları ve araçların kolay entegrasyonu  
Birlikte Çalışabilirlik: Farklı yapay zeka çerçeveleri ve ajan platformlarıyla uyumlu  
Sorumlulukların Ayrımı: Yapay zeka mantığı ile harici veri erişimi arasında net bir ayrım  



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belgenin kendi dilindeki hali yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yorumlamalardan sorumlu değiliz.
